# Lagrange Polynomial Interpolation on Uniform, 3D Numerical Grids

## Author: Zachariah Etienne

## This module implements Lagrange polynomial interpolation on uniform, 3D numerical grids, both with and without coefficient precomputation.

### Python module associated with this tutorial: [interpolation/interpolate_Ccodegen_library.py](../edit/interpolation/interpolate_Ccodegen_library.py)

<font color='green'>**This module has been validated to generate codes that exhibit convergence to zero of interpolation errors at the expected convergence rates. We also confirm below that this notebook generates identical codes to the associated Python module [interpolation/interpolate_Ccodegen_library.py](../edit/interpolation/interpolate_Ccodegen_library.py).**</font>

**Validation notes**: See [Tutorial-Start_to_Finish-Interpolation_from_Uniform_Grid-3D.ipynb
](Tutorial-Start_to_Finish-Interpolation_from_Uniform_Grid-3D.ipynb) for convergence test results.
## Introduction:

We wish to implement the following straightforward algorithm to interpolate data from a uniform grid to a given destination point $(x^0,x^1,x^2)$

$$
P_{N_{0},N_{1},N_{2}}(x^0,x^1,x^2) =
\sum_{i=0}^{N_{0}} \sum_{j=0}^{N_{1}} \sum_{k=0}^{N_{2}} 
\bar{l}^{0}_i \bar{m}^{1}_j \bar{n}^{2}_k \cdot f(x^0_i,x^1_j,x^2_k),
$$
where $\bar{l}^{0}_i$, $\bar{m}^{1}_j$, and $\bar{n}^{2}_k$ are given by
$$
\bar{l}^{0}_i = \prod_{\begin{smallmatrix}0\le l \le N_{0}, \\ l\ne i\end{smallmatrix}} 
\frac{x^0 - x^0_l}{x^0_i-x^0_l}, \quad
\bar{m}^{1}_j = \prod_{\begin{smallmatrix}0\le m \le N_{1}, \\ m\ne j\end{smallmatrix}} 
\frac{x^1 - x^1_m}{x^1_j-x^1_m}, \quad
\bar{n}^{2}_k = \prod_{\begin{smallmatrix}0\le n \le N_{2}, \\ n\ne k\end{smallmatrix}} 
\frac{x^2 - x^2_n}{x^2_k-x^2_n}.
$$
On uniform grids, the denominators containing terms like, $x^0_i-x^0_l$ can be simplified further, as
$$
x^0_i = x^0_{\rm min} + i \Delta x^0,
$$
so that
$$
x^0_i-x^0_l = (i-l) \Delta x^0,
$$
which enables us to rewrite the above formula for $\bar{l}^0_i$ as follows:
\begin{align}
\bar{l}^{0}_i &= \prod_{\begin{smallmatrix}0\le l \le N_{0}, \\ l\ne i\end{smallmatrix}} \frac{x^0-x^0_l}{x^0_i-x^0_l} \\
&= \left(\Delta x^0\right)^{-N_0} \prod_{\begin{smallmatrix}0\le l \le N_{0}, \\ l\ne i\end{smallmatrix}} \frac{x^0-x^0_l}{i-l} \\
&= \left(\Delta x^0\right)^{-N_0} 
\underbrace{\left[\prod_{\begin{smallmatrix}0\le l \le N_{0}, \\ l\ne i\end{smallmatrix}} \left(x^0-x^0_l\right)\right]}_{l^0_i}
\underbrace{\left[\prod_{\begin{smallmatrix}0\le l \le N_{0}, \\ l\ne i\end{smallmatrix}} \frac{1}{i-l}\right]}_{\left(w^0_i\right)^{-1}} \\
&= \left(\Delta x^0\right)^{-N_0} l^0_i \left(w^0_i\right)^{-1}.
\end{align}
Splitting off the integer arithmetic needed to compute $w^0_i$, we gain a slight performance boost. Similarly, we rewrite $\bar{m}^1_j$ and $\bar{n}^2_k$ as
\begin{align}
\bar{m}^{0}_j &= \left(\Delta x^1\right)^{-N_1} m^1_j \left(w^1_j\right)^{-1},\\
\bar{n}^{1}_k &= \left(\Delta x^2\right)^{-N_2} n^2_k \left(w^2_k\right)^{-1}.
\end{align}

Putting the above together, we get
\begin{align}
P_{N_{0},N_{1},N_{2}}(x^0,x^1,x^2) &=
\left(\Delta x^0\right)^{-N_0}
\left(\Delta x^1\right)^{-N_1}
\left(\Delta x^2\right)^{-N_2}
\sum_{i=0}^{N_{0}} \sum_{j=0}^{N_{1}} \sum_{k=0}^{N_{2}} 
\left[l^{0}_i \left(w^0_i\right)^{-1}\right] 
\left[m^{1}_j \left(w^1_j\right)^{-1}\right]
\left[n^{2}_k \left(w^2_k\right)^{-1}\right] \cdot f(x^0_i,x^1_j,x^2_k).
\end{align}

Below we split the computation of the above expression into C functions, and validate against the Python module [interpolation/interpolate_Ccodegen_library.py](../edit/interpolation/interpolate_Ccodegen_library.py).

<a id='toc'></a>

# Table of Contents:  
$$\label{toc}$$

1. [Step 1](#interpcoeffs): `Lagrange_interp_coeffs_3D()`: Compute the interpolation coefficients `l0i__times__w0i_inv`=$\left[l^{0}_i \left(w^0_i\right)^{-1}\right]$, etc. inside the triple sum
1. [Step 2](#triplesum): `Lagrange_sum_3D()`: Evaluate the triple sum
1. [Step 3](#interpstencilindices): `construct_x0i_x1j_x2k_stencil_arrays_3D()`: Driver function for constructing interpolation stencil indices `x0i_idx[N0]`, `x1j_idx[N1]`, and `x2k_idx[N2]`
1. [Step 4](#computep): `uniform_Lagrange_interp_3D()`: Compute $P_{N_{0},N_{1},N_{2}}(x^0,x^1,x^2)$ at an arbitrary list of points `list_of_interp_pts_x012`
1. [Step 5](#code_validation): Code Validation against `interpolation.interpolate_Ccodegen_library` NRPy+ module
1. [Step 6](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF file

<a id='interpcoeffs'></a>

# Step 1: `Lagrange_interp_coeffs_3D()`: Compute the interpolation coefficients `l0i__times__w0i_inv`=$\left[l^{0}_i \left(w^0_i\right)^{-1}\right]$, etc. inside the triple sum \[Back to [top](#toc)\]
$$\label{interpcoeffs}$$

Recall the triple sum is given by 
$$
\sum_{i=0}^{N_{0}} \sum_{j=0}^{N_{1}} \sum_{k=0}^{N_{2}} 
\left[l^{0}_i \left(w^0_i\right)^{-1}\right] 
\left[m^{1}_j \left(w^1_j\right)^{-1}\right]
\left[n^{2}_k \left(w^2_k\right)^{-1}\right] \cdot f(x^0_i,x^1_j,x^2_k).
$$

Notice 

* the interpolation coefficients $\left[l^{0}_i \left(w^0_i\right)^{-1}\right]$ depend only on $i$, 
* the interpolation coefficients $\left[m^{1}_j \left(w^1_j\right)^{-1}\right]$ depend only on $j$, and
* the interpolation coefficients $\left[n^{2}_k \left(w^2_k\right)^{-1}\right]$ depend only on $k$, 

so they can be precomputed and stored.

Basically this function computes $l^0_i \times \left(w^0_i\right)^{-1}$, $m^{1}_j \times \left(w^1_j\right)^{-1}$ , and $n^{2}_k \times \left(w^2_k\right)^{-1}$ as derived above, where e.g.,
\begin{equation}
l^0_i \times \left(w^0_i\right)^{-1} = 
\underbrace{\left[\prod_{\begin{smallmatrix}0\le l \le N_{0}, \\ l\ne i\end{smallmatrix}} \left(x^0-x^0_l\right)\right]}_{l^0_i}
\underbrace{\left[\prod_{\begin{smallmatrix}0\le l \le N_{0}, \\ l\ne i\end{smallmatrix}} \frac{1}{i-l}\right]}_{\left(w^0_i\right)^{-1}}
\end{equation}

In [1]:
def indent_Ccode(indent, Ccode):
    Ccodesplit = Ccode.splitlines()
    outstring = ""
    for i in range(len(Ccodesplit)):
        outstring += indent + Ccodesplit[i] + '\n'
    return outstring
def desc_format(desc):
    return "/*\n" + indent_Ccode(" * ", desc) + " */\n"
# We just return a string containing the function so the function can be more easily inlined.
#  Otherwise (just_return_string=False) will result in function being placed in its own C file.
def codegen_Lagrange_interp_coeffs_3D(just_return_string=True):
    includes = ["NRPy_basic_defines.h", # // Defines REAL, NinterpGHOSTS, NGHOSTS, & interpstruct. This header is generated in NRPy+.
                "stdio.h", "stdlib.h", "math.h"]
    desc = "Sets up interpolation coefficients at a given point"
    name = "Lagrange_interp_coeffs_3D"
    params = """const int N0,const int N1,const int N2,
                               const REAL *restrict x0i,const REAL *restrict x1j,const REAL *restrict x2k,
                               const REAL x012[3],
                               REAL *restrict l0i__times__w0i_inv,REAL *restrict m1j__times__w1j_inv,REAL *restrict n2k__times__w2k_inv"""
    body = r"""
// Computing the denominator using integer operations in this way
//     yields 6% overall speedup, with 128^3 grid,
//     num_interp_pts=1e7, and poly interp order=4
// Total cost: [INTEGER: (N0 + N1 + N2) * (1 assign + 1 mul + 1 sub) + 6 assigns] + 3 typecasts
for(int i=0;i<=N0;i++) {
    REAL prod_numer_i = 1.0; // l0i
    int  prod_denom_i = 1;   // w0i
    for(int l=0;  l<i;  l++) { prod_denom_i *= i-l; prod_numer_i *= x012[0] - x0i[l]; }
    for(int l=i+1;l<=N0;l++) { prod_denom_i *= i-l; prod_numer_i *= x012[0] - x0i[l]; }
    //                         [       w0i        ] [            l0i                ]
    l0i__times__w0i_inv[i] = prod_numer_i / ( (REAL)prod_denom_i );
}
for(int j=0;j<=N1;j++) {
    REAL prod_numer_j = 1.0; // m1j
    int  prod_denom_j = 1;   // w1j
    for(int m=0;  m<j;  m++) { prod_denom_j *= j-m; prod_numer_j *= x012[1] - x1j[m]; }
    for(int m=j+1;m<=N1;m++) { prod_denom_j *= j-m; prod_numer_j *= x012[1] - x1j[m]; }
    //                         [       w1j        ] [            m1j                ]
    m1j__times__w1j_inv[j] = prod_numer_j / ( (REAL)prod_denom_j );
}
for(int k=0;k<=N2;k++) {
    REAL prod_numer_k = 1.0; // n2k
    int  prod_denom_k = 1;   // w2k
    for(int n=0;  n<k;  n++) { prod_denom_k *= k-n; prod_numer_k *= x012[2] - x2k[n]; }
    for(int n=k+1;n<=N2;n++) { prod_denom_k *= k-n; prod_numer_k *= x012[2] - x2k[n]; }
    //                         [       w2k        ] [            n2k                ]
    n2k__times__w2k_inv[k] = prod_numer_k / ( (REAL)prod_denom_k );
}
"""
    body = indent_Ccode('    ', body)
    if just_return_string:
        return desc_format(desc) + "void" + " " + name + "(" + params + ") {\n" + body + "}\n"
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name, params=params,
        body=body, enableCparameters=False)

<a id='triplesum'></a>

# Step 2: `Lagrange_sum_3D()`: Evaluate the triple sum \[Back to [top](#toc)\]
$$\label{triplesum}$$

Recall the triple sum (**excluding the prefactor** $\left(\Delta x^0\right)^{-N_0}
\left(\Delta x^1\right)^{-N_1}
\left(\Delta x^2\right)^{-N_2}$) is given by
$$
\sum_{i=0}^{N_{0}} \sum_{j=0}^{N_{1}} \sum_{k=0}^{N_{2}} 
\left[\left(w^{0}_i\right)^{-1} (x^0-x^0_i)\right]^{-1} 
\left[\left(w^{1}_j\right)^{-1} (x^1-x^1_j)\right]^{-1}
\left[\left(w^{2}_k\right)^{-1} (x^2-x^2_k)\right]^{-1}
f(x^0_i,x^1_j,x^2_k),
$$
where the interpolation coefficients inside the triple sum are computed in the `interp_coeffs_Lagrange_3D()` function documented above. `Lagrange_sum_3D()` defined below computes this triple sum:

In [2]:
# We just return a string containing the function so the function can be more easily inlined.
#  Otherwise (just_return_string=False) will result in function being placed in its own C file.
def codegen_Lagrange_sum_3D(just_return_string=True):
    includes = ["NRPy_basic_defines.h", # // Defines REAL, NinterpGHOSTS, NGHOSTS, & interpstruct. This header is generated in NRPy+.
                "stdio.h", "stdlib.h", "math.h"]
    desc = """Computes 3D sum for interpolation at a single point.
    Note that this function does NOT multiply by the requisite (Delta x_i)^(N_i)"""
    type = "inline REAL"
    name = "Lagrange_sum_3D"
    params = """const int N0,const int N1,const int N2,
    const REAL *restrict l0i__times__w0i_inv,
    const REAL *restrict m1j__times__w1j_inv,
    const REAL *restrict n2k__times__w2k_inv,
    const REAL *restrict f"""
    body = r"""
// Now perform sum, with a total cost of
//  N0N1 * (1 assignment, 1 multiply [1 + 1 ~ 2 FLOPs])
//   + N0N1N2 * ( 1 assignment, 1 add, 2 multiplies [1 + 1 + 2 ~ 4 FLOPs])
REAL sum = 0.0;
int idx = 0;
for(int i=0;i<=N0;i++) {
    for(int j=0;j<=N1;j++) {
        // (N0+1)(N1+1) * (1 assignment, 1 multiply):
        const REAL l0i_term_times_m1j_term = l0i__times__w0i_inv[i]*m1j__times__w1j_inv[j];
        for(int k=0;k<=N2;k++) {
            // (N0+1)(N1+1)(N2+1) * ( 1 assignment, 1 add, 2 multiplies ):
            sum += f[idx] * ( l0i_term_times_m1j_term *  n2k__times__w2k_inv[k] );
            idx++;
        }
    }
}
return sum;
"""
    body = indent_Ccode('    ', body)
    if just_return_string:
        return desc_format(desc) + type + " " + name + "(" + params + ") {\n" + body + "}\n"
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        type=type,     name=name,     params=params,
        body=body, enableCparameters=False)

<a id='interpstencilindices'></a>

# Step 3: `construct_x0i_x1j_x2k_stencil_arrays_3D()`: Driver function for constructing interpolation stencil indices `x0i_idx[N0]`, `x1j_idx[N1]`, and `x2k_idx[N2]` \[Back to [top](#toc)\]
$$\label{interpstencilindices}$$

This function, `construct_x0i_x1j_x2k_stencil_arrays_3D()`, takes as input the desired $(x^0,x^1,x^2)$ and outputs integer arrays `x0i_idx[N0]`, `x1j_idx[N1]`, and `x2k_idx[N2]` corresponding to the indices of each gridpoint in the interpolation stencil. 

When dealing with uniform grids, we prefer that the interpolation stencil is as close to $(x^0,x^1,x^2)$ as possible. For a uniform grid in the $x^0$ direction, by definition
$$
\text{x0i[i0]} = \text{x0i[0]} + i0*(\text{x0i[1]} - \text{x0i[0]}).
$$

However, the interpolation destination point $x^0$ will generally not lie exactly on a gridpoint, so we choose to center the interpolation stencil as close as possible onto the point $x^0$. The non-integer, real-valued index $\text{i0}_{\rm real}$ corresponding to point $x^0=\text{x0i[i0}_{\rm real}\text{]}$ is found via
$$
x^0 = \text{x0i[0]} + i0_{\rm close}*(\text{x0i[1]} - \text{x0i[0]}),
$$
so $\text{i0}_{\rm real}$ is given by
$$
\text{i0}_{\rm real} = \frac{x^0 - \text{x0i[0]}}{\text{x0i[1]} - \text{x0i[0]}},
$$
and the *integer* index corresponding to the closest sampled gridpoint $\text{i0}_{\rm int}$ is given by
$$
\text{i0}_{\rm int} = \text{int}\left(\frac{x^0 - \text{x0i[0]}}{\text{x0i[1]} - \text{x0i[0]}} + \frac{1}{2}\right),
$$
where $\text{int}()$ is the usual float-to-integer typecast, which always rounds down (i.e., $\text{int}(4.99)=4$, so the factor of $\frac{1}{2}$ is added to ensure more appropriate rounding.

Using the above formula to compute $(\text{i0}_{\rm int},\text{i1}_{\rm int},\text{i2}_{\rm int})$, the following function outputs $({\rm x0i\_idx[]},{\rm x1j\_idx[]},{\rm x2k\_idx[]})$ arrays, which store the interpolation stencil on the uniform numerical grid.

In [3]:
# We just return a string containing the function so the function can be more easily inlined.
#  Otherwise (just_return_string=False) will result in function being placed in its own C file.
def codegen_construct_x0i_x1j_x2k_stencil_arrays_3D(just_return_string=True):
    includes = ["NRPy_basic_defines.h", # // Defines REAL, NinterpGHOSTS, NGHOSTS, & interpstruct. This header is generated in NRPy+.
                "stdio.h", "stdlib.h", "math.h"]
    desc = """Construct integer arrays x0i_idx[N0],x1j_idx[N1],x2k_idx[N2] """
    name = "construct_x0i_x1j_x2k_stencil_arrays_3D"
    params = """const int Nxx_plus_2NGHOSTS0,const int Nxx_plus_2NGHOSTS1,const int Nxx_plus_2NGHOSTS2,  REAL *restrict xx[3],
    const REAL x012[3],
    const int N0,const int N1,const int N2,
    int *restrict x0i_idx,int *restrict x1j_idx,int *restrict x2k_idx"""
    body = r"""
// Total cost: 3 * (2 subs + 1 div + 1 add) + 3 * (1 integer sub, div, and add)
    const int i012_int[3] = { (int)( (x012[0] - xx[0][0])/(xx[0][1] - xx[0][0]) + 0.5 ),
                              (int)( (x012[1] - xx[1][0])/(xx[1][1] - xx[1][0]) + 0.5 ),
                              (int)( (x012[2] - xx[2][0])/(xx[2][1] - xx[2][0]) + 0.5 ) };
//#define BOUNDS_CHECKING
#ifndef BOUNDS_CHECKING
    for(int i0=0;i0<=N0;i0++) x0i_idx[i0] = i012_int[0] - N0/2 + i0;
    for(int i1=0;i1<=N1;i1++) x1j_idx[i1] = i012_int[1] - N1/2 + i1;
    for(int i2=0;i2<=N2;i2++) x2k_idx[i2] = i012_int[2] - N2/2 + i2;
#else
    //printf("%d %d %d | %d iiii\n", i012_int[0], i012_int[1], i012_int[2], Nxx_plus_2NGHOSTS0);
    for(int i0=0;i0<=N0;i0++) {
        x0i_idx[i0] = i012_int[0] - N0/2 + i0;
        if(x0i_idx[i0] < 0 || x0i_idx[i0] >= Nxx_plus_2NGHOSTS0) {
            printf("ERROR: index %d is out of range!\n",x0i_idx[i0]); exit(1);
        }
    }
    for(int i1=0;i1<=N1;i1++) {
        x1j_idx[i1] = i012_int[1] - N1/2 + i1;
        if(x1j_idx[i1] < 0 || x1j_idx[i1] >= Nxx_plus_2NGHOSTS1) {
            printf("ERROR: index %d is out of range!\n",x1j_idx[i1]); exit(1);
        }
    }
    for(int i2=0;i2<=N2;i2++) {
        x2k_idx[i2] = i012_int[2] - N2/2 + i2;
        if(x2k_idx[i2] < 0 || x2k_idx[i2] >= Nxx_plus_2NGHOSTS2) {
            printf("ERROR: index %d is out of range!\n",x2k_idx[i2]); exit(1);
        }
    }
#endif
"""
    body = indent_Ccode('    ', body)
    if just_return_string:
        return desc_format(desc) + "void" + " " + name + "(" + params + ") {\n" + body + "}\n"

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name,     params=params,
        body=body, enableCparameters=False)

<a id='computep'></a>

# Step 4: `uniform_Lagrange_interp_3D()`: Compute $P_{N_{0},N_{1},N_{2}}(x^0,x^1,x^2)$ at an arbitrary list of points `list_of_interp_pts_x012` \[Back to [top](#toc)\]
$$\label{computep}$$

As shown above, the 3D Lagrange polynomial interpolation formula can be written as
$$
P_{N_{0},N_{1},N_{2}}(x^0,x^1,x^2) =
\left(\Delta x^0\right)^{-N_0}
\left(\Delta x^1\right)^{-N_1}
\left(\Delta x^2\right)^{-N_2}
\sum_{i=0}^{N_{0}} \sum_{j=0}^{N_{1}} \sum_{k=0}^{N_{2}} 
\left[l^{0}_i \left(w^0_i\right)^{-1}\right] 
\left[m^{1}_j \left(w^1_j\right)^{-1}\right]
\left[n^{2}_k \left(w^2_k\right)^{-1}\right] \cdot f(x^0_i,x^1_j,x^2_k).
$$

In [4]:
def add_uniform_Lagrange_interp_3D_to_Cfunction_dict(prefunc=""):
    includes = ["NRPy_basic_defines.h", # // Defines REAL, NinterpGHOSTS, NGHOSTS, & interpstruct. This header is generated in NRPy+.
                "stdio.h", "stdlib.h", "math.h"]
    prefunc += """// The following will result in cache misses just below since the innermost loop is
//     over gf, but it ensures consistency with IDX4() in the rest of NRPy+,
//     and ensures quick access with minimal cache misses when accessed from elsewhere.
#define INTERP_IDX(num_interp_pts,gf,pt) ( (pt) + (num_interp_pts) * (gf) )
"""
    desc = "uniform_Lagrange_interp_3D(): interpolate to an arbitrary list of points list_of_interp_pts_x012"
    name = "uniform_Lagrange_interp_3D"
    params = """const int Nxx_plus_2NGHOSTS0,const int Nxx_plus_2NGHOSTS1,const int Nxx_plus_2NGHOSTS2,  REAL *restrict xx[3],const REAL *dx012_term_inv,

    const REAL *restrict in_gfs,
    const int num_interp_gfs, const int list_of_interp_gfs[num_interp_gfs],
    const int num_interp_pts, const REAL *restrict list_of_interp_pts_x0, const REAL *restrict list_of_interp_pts_x1, const REAL *restrict list_of_interp_pts_x2,

    const int N0,const int N1,const int N2,

    REAL *restrict interp_output"""
    body = r"""REAL *restrict f;
int *restrict x0i_idx;
int *restrict x1j_idx;
int *restrict x2k_idx;
REAL *restrict x0i;
REAL *restrict x1j;
REAL *restrict x2k;
REAL *restrict l0i__times__w0i_inv;
REAL *restrict m1j__times__w1j_inv;
REAL *restrict n2k__times__w2k_inv;
#pragma omp parallel private(f,x0i_idx,x1j_idx,x2k_idx,x0i,x1j,x2k,l0i__times__w0i_inv,m1j__times__w1j_inv,n2k__times__w2k_inv)
{
    f = (REAL *)malloc(sizeof(REAL)*(N0+1)*(N1+1)*(N2+1));
    x0i_idx = (int *)malloc(sizeof(int)*(N0+1));
    x1j_idx = (int *)malloc(sizeof(int)*(N1+1));
    x2k_idx = (int *)malloc(sizeof(int)*(N2+1));
    x0i = (REAL *)malloc(sizeof(REAL)*(N0+1));
    x1j = (REAL *)malloc(sizeof(REAL)*(N1+1));
    x2k = (REAL *)malloc(sizeof(REAL)*(N2+1));
    l0i__times__w0i_inv = (REAL *)malloc(sizeof(REAL)*(N0+1));
    m1j__times__w1j_inv = (REAL *)malloc(sizeof(REAL)*(N1+1));
    n2k__times__w2k_inv = (REAL *)malloc(sizeof(REAL)*(N2+1));

    #pragma omp for
    for(int interp_pt = 0; interp_pt<num_interp_pts; interp_pt++) {
        const REAL curr_xx[3] = { list_of_interp_pts_x0[interp_pt],
                                  list_of_interp_pts_x1[interp_pt],
                                  list_of_interp_pts_x2[interp_pt] };

        // Total cost: 3 * (2 subs + 1 div + 1 add) + 3 * (1 integer sub, div, and add)
        {
            construct_x0i_x1j_x2k_stencil_arrays_3D(
                Nxx_plus_2NGHOSTS0,Nxx_plus_2NGHOSTS1,Nxx_plus_2NGHOSTS2,xx,
                curr_xx,
                N0,N1,N2, x0i_idx,x1j_idx,x2k_idx);
        }

        // Total cost: (N0+N1+N2) * 1 assignment
        {
            for(int i0=0;i0<=N0;i0++) x0i[i0] = xx[0][x0i_idx[i0]];
            for(int i1=0;i1<=N1;i1++) x1j[i1] = xx[1][x1j_idx[i1]];
            for(int i2=0;i2<=N2;i2++) x2k[i2] = xx[2][x2k_idx[i2]];
        }

        // Total cost: [INTEGER: (N0-1 + N1-1 + N2-1) * (1 assign + 1 mul + 1 sub) + 6 assigns] + 3 typecasts
        //   PLUS (N0 + N1 + N2) * (1 assignment, 1 divide, 1 multiply, and 1 subtract [1 + 3 + 1 + 1 ~ 6 FLOPs])
        Lagrange_interp_coeffs_3D(
                N0,N1,N2, x0i,x1j,x2k, curr_xx,
                l0i__times__w0i_inv,m1j__times__w1j_inv,n2k__times__w2k_inv);

        for(int gf=0;gf<num_interp_gfs;gf++) {
            const int which_gf = list_of_interp_gfs[gf];
            // Total cost: N0N1N2 assignments
            int iii=0;
            for(int i0=0;i0<=N0;i0++) for(int i1=0;i1<=N1;i1++) for(int i2=0;i2<=N2;i2++) {
                  f[iii] = in_gfs[IDX4S(which_gf,x0i_idx[i0],x1j_idx[i1],x2k_idx[i2])];
                  iii++;
                }
            interp_output[INTERP_IDX(num_interp_pts, gf,interp_pt)] =
                (*dx012_term_inv)*
                Lagrange_sum_3D(N0,N1,N2,
                                l0i__times__w0i_inv,
                                m1j__times__w1j_inv,
                                n2k__times__w2k_inv,
                                f);
        }
    }
    free(f);
    free(x0i_idx);
    free(x1j_idx);
    free(x2k_idx);
    free(x0i);
    free(x1j);
    free(x2k);
    free(l0i__times__w0i_inv);
    free(m1j__times__w1j_inv);
    free(n2k__times__w2k_inv);
}
"""
    body = indent_Ccode('    ', body)
    add_to_Cfunction_dict(
        includes=includes,
        prefunc=prefunc,
        desc=desc,
        name=name, params=params,
        body=body, enableCparameters=False)

<a id='code_validation'></a>

# Step 5: Code Validation against `interpolation.interpolate_Ccodegen_library` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the C code generated by

1. this tutorial and 
2. the [NRPy+ interpolation.interpolate_Ccodegen_library](../edit/interpolation/interpolate_Ccodegen_library.py) module.

In [5]:
from outputC import add_to_Cfunction_dict, outC_function_dict
inlined_functions  = codegen_Lagrange_interp_coeffs_3D()
inlined_functions += codegen_Lagrange_sum_3D()
inlined_functions += codegen_construct_x0i_x1j_x2k_stencil_arrays_3D()

add_uniform_Lagrange_interp_3D_to_Cfunction_dict(prefunc = inlined_functions + "\n\n")

Jupyter_notebook_result = outC_function_dict['uniform_Lagrange_interp_3D']

outC_function_dict = {}

from outputC import add_to_Cfunction_dict, outC_function_dict
import interpolation.interpolate_Ccodegen_library as interpClib

inlined_functions  = interpClib.codegen_Lagrange_interp_coeffs_3D()
inlined_functions += interpClib.codegen_Lagrange_sum_3D()
inlined_functions += interpClib.codegen_construct_x0i_x1j_x2k_stencil_arrays_3D()
interpClib.add_uniform_Lagrange_interp_3D_to_Cfunction_dict(prefunc = inlined_functions + "\n\n")
python_module_result = outC_function_dict['uniform_Lagrange_interp_3D']

import sys
# Could also use difflib here, but I found errors were harder to parse.
python_module_result_lines = python_module_result.splitlines()
for linenum, line in enumerate(Jupyter_notebook_result.splitlines()):
    if line != python_module_result_lines[linenum]:
        print("Error: Found diff at line " + str(linenum) + ":")
        print("Jupyter notebook:\n" + line)
        print("Python module:\n" + python_module_result_lines[linenum])
        sys.exit(1)
print("No difference. TEST PASSED!")

OUCH! Found uniform_Lagrange_interp_3D in outC_function_master_list.
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 6: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Interpolation-Lagrange_Polynomial-from_Uniform_Grid-3D.pdf](Tutorial-Interpolation-Lagrange_Polynomial-from_Uniform_Grid-3D.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [6]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Interpolation-Lagrange_Polynomial-from_Uniform_Grid-3D")

Created Tutorial-Interpolation-Lagrange_Polynomial-
    from_Uniform_Grid-3D.tex, and compiled LaTeX file to PDF file Tutorial-
    Interpolation-Lagrange_Polynomial-from_Uniform_Grid-3D.pdf
